**Task 1: Data Exploration and Enrichment** Load the starter dataset and enrich it with external high-frequency data (e.g., EthioTelecom reports) as per your specific requirements.

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

In [31]:
try:
    df = pd.read_csv('/content/ethiopia_fi_unified_data.csv')
    refs = pd.read_csv('/content/reference_codes - reference_codes.csv')
    print("✅ All datasets loaded successfully!")
except FileNotFoundError:
    print("Error: Files not found. Please ensure they are in the working directory.")

✅ All datasets loaded successfully!


In [32]:
print(f"Total records: {len(df)}")
print(f"Record Types: {df['record_type'].unique()}")

Total records: 43
Record Types: ['observation' 'target' 'event']


In [33]:
observations = df[df['record_type'] == 'observation']
print("Years covered:", sorted(pd.to_datetime(observations['observation_date']).dt.year.unique()))

Years covered: [np.int32(2014), np.int32(2017), np.int32(2021), np.int32(2023), np.int32(2024), np.int32(2025)]


In [34]:
print("\nTop Indicator Counts:")
print(observations['indicator'].value_counts().head())


Top Indicator Counts:
indicator
Account Ownership Rate         6
Fayda Digital ID Enrollment    3
Mobile Money Account Rate      2
4G Population Coverage         2
P2P Transaction Count          2
Name: count, dtype: int64


In [35]:
real_enrichment_data = [
    # --- OBSERVATIONS: INFRASTRUCTURE ---
    {
        "record_type": "observation",
        "pillar": "Infrastructure",
        "indicator": "Mobile Penetration",
        "indicator_code": "ACC_MOBILE_PEN",
        "value_numeric": 53.5,
        "value_type": "percentage",
        "observation_date": pd.Timestamp("2023-01-01"),
        "source_name": "DataReportal Digital 2023",
        "confidence": "High",
        "notes": "66.8 million connections"
    },
    {
        "record_type": "observation",
        "pillar": "Infrastructure",
        "indicator": "Mobile Penetration",
        "indicator_code": "ACC_MOBILE_PEN",
        "value_numeric": 60.4,
        "value_type": "percentage",
        "observation_date": pd.Timestamp("2024-01-01"),
        "source_name": "DataReportal Digital 2024",
        "confidence": "High",
        "notes": "77.4 million connections (+12% YoY)"
    },
    # --- OBSERVATIONS: USAGE (Telebirr Actuals) ---
    {
        "record_type": "observation",
        "pillar": "Usage",
        "indicator": "Telebirr Registered Users",
        "indicator_code": "USG_TELEBIRR_USERS",
        "value_numeric": 27200000,
        "value_type": "count",
        "observation_date": pd.Timestamp("2023-01-01"),
        "source_name": "EthioTelecom H1 Report",
        "confidence": "High",
        "notes": "27.2M users reported in Jan 2023"
    },
    {
        "record_type": "observation",
        "pillar": "Usage",
        "indicator": "Telebirr Registered Users",
        "indicator_code": "USG_TELEBIRR_USERS",
        "value_numeric": 47550000,
        "value_type": "count",
        "observation_date": pd.Timestamp("2024-06-30"),
        "source_name": "EthioTelecom Annual Report 2023/24",
        "confidence": "High",
        "notes": "Massive growth to 47.55M users"
    },
    # --- EVENTS: MAJOR MARKET SHIFTS ---
    {
        "record_type": "event",
        "category": "product_launch",
        "pillar": "Infrastructure",
        "indicator": "Safaricom M-Pesa Launch", # Adjusted to match schema 'indicator' or 'event_name'
        "indicator_code": "EVT_MPESA",
        "observation_date": pd.Timestamp("2023-08-16"),
        "source_name": "Safaricom Press Release",
        "notes": "M-Pesa goes live, introducing first major competition to Telebirr"
    }
]

In [36]:
new_data_df = pd.DataFrame(real_enrichment_data)

In [37]:
for col in df.columns:
    if col not in new_data_df.columns:
        new_data_df[col] = pd.NA

# Reorder columns to match original
new_data_df = new_data_df[df.columns]

# Concatenate
enriched_df = pd.concat([df, new_data_df], ignore_index=True)

In [38]:
enriched_df['observation_date'] = pd.to_datetime(enriched_df['observation_date'], errors='coerce')
enriched_df['year'] = enriched_df['observation_date'].dt.year

print(f"\nOriginal Row Count: {len(df)}")
print(f"Enriched Row Count: {len(enriched_df)}")


Original Row Count: 43
Enriched Row Count: 48


In [39]:
print("\n--- New Data Verification ---")
display_cols = ['pillar', 'indicator', 'value_numeric', 'observation_date', 'source_name']
print(enriched_df[enriched_df['source_name'].str.contains("DataReportal", na=False)][display_cols])


--- New Data Verification ---
            pillar                        indicator  value_numeric  \
10          ACCESS  Mobile Subscription Penetration           61.4   
43  Infrastructure               Mobile Penetration           53.5   
44  Infrastructure               Mobile Penetration           60.4   

   observation_date                source_name  
10       2025-12-31  DataReportal Digital 2026  
43       2023-01-01  DataReportal Digital 2023  
44       2024-01-01  DataReportal Digital 2024  


In [40]:
enriched_df.to_csv('ethiopia_fi_enriched.csv', index=False)
print(f"\n✅ Success! Enriched data saved to ethiopia_fi_enriched.csv")


✅ Success! Enriched data saved to ethiopia_fi_enriched.csv
